# Movie/Tv Show Recommendation System

## Introduction
- Recommender systems (RSs) are everywhere. Amazon, Netflix, Spotify, YouTube, and many more services and apps we use every day have in the backend some sort of recommendation engine.

- RSs help users to find items they are interested in, and this can increase engagement on the platform: if a platform suggests items of interest, users will spend more time on that platform.

- There are many techniques and strategies to build modern and powerful RSs based on specific application domains
- I am using the content based techniques for building this system.

## So, in this kernel, we'll be building a Movie/ TV Show Recommendation System using those datasets:

- Paramount+ Movies and TV Shows
- Disney+ Movies and TV Shows
- AppleTV+ Movies and TV Shows
- Amazon Prime Movies and TV Shows
- HBO Max Movies and TV Shows
- Netflix Movies and TV Shows

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import missingno as msno
%matplotlib inline

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
#pip install missingno

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Read in the datas
amazon_titles = pd.read_csv("/kaggle/input/amazon-prime-movies-and-tv-shows/titles.csv")
appletv_titles = pd.read_csv("/kaggle/input/apple-tv-movies-and-tv-shows/titles.csv")
disney_titles = pd.read_csv("/kaggle/input/disney-movies-and-tv-shows/titles.csv")
hbo_titles = pd.read_csv("/kaggle/input/hbo-max-movies-and-tv-shows/titles.csv")
netflix_titles = pd.read_csv("/kaggle/input/netflix-movies-and-tv-shows/titles.csv")
paramount_titles = pd.read_csv("/kaggle/input/paramount-movies-and-tv-shows/titles.csv")